In [1]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Conv2DTranspose, Concatenate
from tensorflow.keras import Input
from tensorflow.keras.models import Model

In [2]:
# Repeated convolutional blocks can be defined using a reusable function.
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding='same')(input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(num_filters, 3, padding='same')(input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    return x

In [3]:
# Repeated encoder convolutional blocks can be defined using a reusable function.
def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p


# Repeated decoder blocks can be defined using a reusable function.
def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding='same')(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters=2)
    return x


# S is the output of the conv block --> P is the output of the max pooling --> d is the ouput of the decoder block
def unet_arch(input_shape):
    inputs = Input(input_shape)

    # Following the U-net Architecture
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding='same', activation='sigmoid')(d4)

    model = Model(inputs, outputs, name='U-net')
    return model

In [4]:
input_shape = (512, 512, 3)
model = unet_arch(input_shape)
model.summary()

Model: "U-net"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 512, 512, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 512, 512, 64) 0           batch_normalization_1[0][0]      
______________________________________________________________________________________________

# DATASET

In [5]:
import numpy as np
import os
from glob import glob
from sklearn.model_selection import train_test_split
import cv2
import tensorflow as tf

In [6]:
def image_ops(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (256, 256))
    img = img / 255.0
    img = img.astype(np.float32)
    return img


def mask_ops(path):
    mask = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (256, 256))
    mask = mask.astype(np.float32)
    mask = np.expand_dims(mask, axis=-1)  # (256,256,1)
    return mask


def load_data(path):
    images = glob(os.path.join(path, "images/*"))
    masks = glob(os.path.join(path, "masks/*"))

    train_x, test_x = train_test_split(images, test_size=0.2, random_state=42)
    train_y, test_y = train_test_split(masks, test_size=0.2, random_state=42)


    return (train_x, test_x), (train_y, test_y)


def preprocess(image_path, mask_path):
    def f(image_path, mask_path):
        image_path = image_path.decode()
        mask_path = mask_path.decode()

        x = image_ops(image_path)
        y = mask_ops(mask_path)
        return x, y

    image, mask = tf.numpy_function(f, [image_path, mask_path], [tf.float32, tf.float32])
    # image, mask = f(image_path,mask_path)
    image.set_shape([256, 256, 3])
    mask.set_shape([256, 256, 1])

    return image, mask


def tf_dataset(images, masks, batch=10):
    dataset = tf.data.Dataset.from_tensor_slices((images, masks))
    dataset = dataset.shuffle(buffer_size=5000)
    dataset = dataset.map(preprocess)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(2)
    return dataset

In [7]:
path = "../input/person-segmentation/people_segmentation"  # . signifies the current folder
(train_x, test_x), (train_y, test_y) = load_data(path)
print(f"Training: {len(train_x)} - {len(train_y)}  ")
print(f"Testing: {len(test_x)} - {len(test_y)}  ")
train_dataset = tf_dataset(train_x, train_y, batch=12)

Training: 4542 - 4542  
Testing: 1136 - 1136  


In [8]:
import os
import numpy as np
import tensorflow as tf
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import cv2

In [9]:
import tensorflow as tf 
print(tf.__version__)

2.4.1


In [10]:

input_shape = (256, 256, 3)
batch_size = 12
epochs = 10
lr = 1e-4
model_path = "unet.h5"

""" Load the dataset """

train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
test_dataset = tf_dataset(test_x, test_y, batch=batch_size)

""" Model """
model = unet_arch(input_shape)
"""
model.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam,
    metrics=[
        tf.keras.metrics.MeanIoU(num_classes=2),
        tf.keras.metrics.Recall(),
        tf.keras.metrics.Precision()
    ]
)
"""
model.compile(optimizer="Adam",
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(),
                       tf.keras.metrics.FalseNegatives()])

model.summary()



Model: "U-net"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, 256, 256, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_19 (BatchNo (None, 256, 256, 64) 256         conv2d_20[0][0]                  
__________________________________________________________________________________________________
activation_19 (Activation)      (None, 256, 256, 64) 0           batch_normalization_19[0][0]     
______________________________________________________________________________________________

In [11]:
model.fit(train_dataset, epochs = epochs, verbose =1)

Epoch 1/10
379/379 [==============================] - 107s 259ms/step - loss: 0.6006 - binary_accuracy: 0.7171 - false_negatives: 38571433.0579
Epoch 2/10
379/379 [==============================] - 64s 169ms/step - loss: 0.5138 - binary_accuracy: 0.7391 - false_negatives: 39011258.0105
Epoch 3/10
379/379 [==============================] - 65s 170ms/step - loss: 0.5095 - binary_accuracy: 0.7417 - false_negatives: 38576332.7711
Epoch 4/10
379/379 [==============================] - 65s 170ms/step - loss: 0.5051 - binary_accuracy: 0.7423 - false_negatives: 38741994.3526
Epoch 5/10
379/379 [==============================] - 65s 172ms/step - loss: 0.5075 - binary_accuracy: 0.7423 - false_negatives: 38565369.0842
Epoch 6/10
379/379 [==============================] - 65s 172ms/step - loss: 0.5054 - binary_accuracy: 0.7426 - false_negatives: 38716396.5395
Epoch 7/10
379/379 [==============================] - 65s 171ms/step - loss: 0.5084 - binary_accuracy: 0.7402 - false_negatives: 38720121.371